In [ ]:
pip install torch torchvision --quiet


In [ ]:
pip install pandas matplotlib scikit-learn --quiet


In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Define transformations: Convert to Tensor and Normalize
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

# Download and load the training dataset
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# Create DataLoader for batching
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [ ]:
import torch.nn as nn
import torch.optim as optim

class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = x.view(-1, 28*28)  # Flatten the input (MNIST images are 28x28 pixels)
        x = torch.relu(self.fc1(x))  # ReLU activation
        x = self.fc2(x)
        return x

# Instantiate the model
model = SimpleNN(input_size=28*28, hidden_size=128, output_size=10)  # 10 classes for MNIST digits


In [ ]:
criterion = nn.CrossEntropyLoss()  # For multi-class classification
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
epochs = 5  # Train for 5 epochs
for epoch in range(epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0

    for images, labels in train_loader:
        optimizer.zero_grad()  # Zero gradients

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}')


In [ ]:
model.eval()  # Set model to evaluation mode
correct = 0
total = 0

with torch.no_grad():  # No need to track gradients during evaluation
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total:.2f}%')


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Example data
data = {'Values': [1, 2, 2, 3, 3, 3, 4, 5]}
df = pd.DataFrame(data)

# Calculate the mode
mode_value = df['Values'].mode()[0]

# Plot the data
plt.figure(figsize=(8, 6))
sns.countplot(x='Values', data=df)
plt.axvline(x=mode_value, color='r', linestyle='--', label=f'Mode: {mode_value}')
plt.legend()
plt.title('Visualization of Mode')
plt.show()
